# DS enrichment

Goal: investigate how different DS filter levels influence enrichment results

## Setup

In [1]:
# Import modules and R
import numpy as np, pandas as pd
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

In [2]:
# Import local code
import sys
sys.path.append('../code')
import pcaVersion
from processing_helpers import *
from analysis_helpers import *
%R source("../code/Rplots.R");
%R source("../code/brainPlots.R");
from brainspan import *

Attaching package: ‘lemon’



    %||%



    CoordCartesian, element_render


